In [1]:
import json
import sys
dirs = ["../..", "../create_prompt_llm", ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag
import covmis

search_engine = "brave"

data_search_dir = f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-2024/data_{search_engine}_search.json"
def load_search():
    with open(data_search_dir, "r") as f:
        data = json.load(f)
    return data

def save_search(data):
    with open(data_search_dir, "w") as f:
        json.dump(data, f, indent=4)

def load_train_bak():
    with open("/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train.json", "r") as f:
        return json.load(f)
    
def load_train():
    with open("/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-2024/data.json", "r") as f:
        return json.load(f)
    
def load_type1():
    with open("output/type_1.json", "r") as f:
        return json.load(f)

def load_type2():
    with open("output/type_2.json", "r") as f:
        return json.load(f)
    
def save_type1(x):
    with open("output/type_1.json", "w") as f:
        json.dump(x, f, indent=4)
        
def save_type2(x):
    with open("output/type_2.json", "w") as f:
        json.dump(x, f, indent=4)

def item_exist(x, data):
    for item in data:
        if item["id"] == x["id"]:
            return True
    return False

def update_data_type1(x, data: list):

    data_type2 = load_type2()
    if item_exist(x, data_type2):
        return
    
    if not item_exist(x, data):
        data.append(x)

def update_data_type2(x, data: list):

    if not item_exist(x, data):
        data.append(x)



排查某些关键词会导致标签错误的情况

In [4]:
import nltk

# 关键词：
false_words = [
    "not", "?", "hoax", "n't", "doctored", "false", "fabricate", 
    "misinformation", "scam", "no", "in fact", "refute", "at least", "old"
]

import pandas as pd

data_train = load_train()
data_type1 = load_type1()
data_type2 = load_type2()

for i, item in enumerate(data_train):
    if item_exist(item, data_type1) or item_exist(item, data_type2):
        continue

    if item["label"] == 0:
        label = "FALSE"
    elif item["label"] == 1:
        label = "PARTLY TRUE"
    elif item["label"] == 2:
        label = "TRUE"
    else:
        raise Exception()
    
    if item["label"] == 1:
        continue

    claim_words = nltk.word_tokenize(item["claim"])
    if len(claim_words) <= 5:
        continue
    
    checked = False
    # 长度<=5已经核查
    # false word 已核查 
    for word in false_words:
        if word in claim_words:
            checked = True
            # print(f'{item["id"]}({label}): {item["claim"]}')
            break
    if checked:
        continue
    if "conspiracy" in claim_words:
        print(f'{item["id"]}({label}): {item["claim"]}')


905(FALSE): There is a conspiracy by the Deep State and China to sweep population control and derail President Trump via coronavirus.
2921(FALSE): A long conspiracy video claims that: vaccines can alter a person’s DNA, nanorobots are inserted with the vaccine to collect biometric data, Bill Gates already owns this data and the body can receive 5G signal after the vaccine is taken
3916(FALSE): Video claims COVID-19 patent has existed for years, hinting to conspiracy
4896(FALSE): Hydroxychloroquine “cures COVID-19” but there is a conspiracy by Biil Gates and Dr. Fauci to ban it.
5791(FALSE): Drinking tea and hot water kills the virus; lemon slices in warm water may save your life; it’s recommended to keep your money in a sealed plastic bag; a conspiracy theory suggests China is benefiting from the pandemic; vinegar is good for cleaning your hands.
6517(FALSE): A conspiracy theory audio that claims, amongst other things, that an Australian vaccine infected people with HIV
7189(FALSE): The

In [2]:
s = "\u201cCoronavirus \u2014 22. Clintons \u2014 39.\u201d"
s2 = "\u201cPresident Trump just announced that the \u2018biological\u2019 lab in Wuhan where the COVID-19 virus was created was \u2018funded\u2019 by President Barak (sp) Hussein Obama in 2015 to the tune of $3,800,000 American dollars!\u201d"





print(s)

“Coronavirus — 22. Clintons — 39.”


删除引号

In [5]:
def find_quotes_positions(text):
    quotes = ["'", '"', '“', '”', '‘', '’']
    stack = []
    positions = []
    unmatched = []
    
    # Define matching pairs for Chinese quotes
    matching_pairs = {'‘': '’', '“': '”'}
    
    # Define cases to ignore
    ignore_cases = ["'s", "’s", "n't", "n’t", "’re", "'re", "’ll", "'ll", 
                    "’m", "'m", "’ve", "'ve", "’d", "'d", "s’", "s'"]
    
    i = 0
    while i < len(text):
        char = text[i]
        
        # Check if the next characters match any ignore case
        ignore = False
        for case in ignore_cases:
            if text[i: i + len(case)] == case:
                if i + len(case) >= len(text) or not text[i + len(case)].isalpha():
                    ignore = True
                    
                    i += len(case) - 1  # Skip the length of the ignore case
                    break
                
        if not ignore and char in quotes:
            # Handle Chinese quotes separately
            if char in matching_pairs.keys():
                stack.append((char, i))
            elif char in matching_pairs.values():
                if stack and matching_pairs[stack[-1][0]] == char:
                    start_position = stack.pop()[1]
                    positions.append((start_position, i))
                else:
                    unmatched.append(i)
            # Handle regular quotes
            elif stack and stack[-1][0] == char:
                start_position = stack.pop()[1]
                positions.append((start_position, i))
            else:
                stack.append((char, i))
                
        i += 1

    # All remaining items in stack are unmatched quotes
    unmatched.extend(pos[1] for pos in stack)

    return positions, unmatched

# 示例字符串
text = "An image of an elderly man crying ‘at the high cost of masks’ outside a pharmacy for the coronavirus in Spain."

text = """“他写道：‘今天是个好日子’，而她回复：“确实如此！”。He said, 'It's a great day.' 
and she replied, 'Yes, it’s fantastic!' She didn’t say she can’t do it. 
They’re going, you’re coming, I’m staying, we’ve done it, you’ll see, he’d known, it’s hers’.”"""

positions, unmatched = find_quotes_positions(text)

print("匹配的引号位置:", positions)
print("未匹配的引号位置:", unmatched)
print((0, len(text) - 1))


匹配的引号位置: [(5, 13), (20, 26), (37, 55), (75, 96), (0, 224)]
未匹配的引号位置: []
(0, 224)


In [25]:
quotation_marks = ["\"", "\'", "”", "“", "’", "‘"]

# 也要忽略’re, 're, ’ll, 'll, ’m, 'm, ’ve, 've, ’d, 'd, s’, s'的情况
# ’re  ’ll ’m ’ve ’d  s’
# I’s almost inarguable that more testing is generating more cases. To one extent or another, the volume of new cases coming in is a reflection of a great success in expanding testing across the country.


data_train = load_train()
data_type1 = load_type1()
data_type2 = load_type2()

for i, item in enumerate(data_train):
    # if item_exist(item, data_type1) or item_exist(item, data_type2):
    #     continue

    # if item["label"] == 0:
    #     label = "FALSE"
    # elif item["label"] == 1:
    #     label = "PARTLY TRUE"
    # elif item["label"] == 2:
    #     label = "TRUE"
    # else:
    #     raise Exception()
    
    
    claim = item["claim"]
    ind = item["id"]
    if ind in []:
        # print()
        continue
    # if not((claim.startswith("\"") or claim.startswith("“")) and (claim.endswith("\"") or claim.endswith("”"))):
    #     continue
    if not ((claim.startswith("“") and claim.endswith("”"))):
        continue
    try:
        matched_quotes, unmatched_quotes = find_quotes_positions(item["claim"])

    except:
        pass
        # print(f"fail: {claim}")
    if len(unmatched_quotes) != 0:
        pass
        # print(f'{ind}: {claim}')
    else:
        if (0, len(claim) - 1) in matched_quotes:
            item["claim"] = item["claim"][1:-1]


In [26]:
# with open("covmis_data/train.json", "w") as f:
#     json.dump(data_train, f, indent=4)


In [11]:
s = "\u201cWe inherited an obsolete deal [for testing] and we've made a good thing out of it. I haven't heard that question in a while, but go ahead.\u201d"

s.endswith("”")

True

排查句子不完整的情况

In [1]:
# import os
# import torch

# os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'


# from swift.llm import (
#     ModelType, get_vllm_engine, get_default_template_type,
#     get_template, inference_vllm, VllmGenerationConfig
# )
# from custom import CustomModelType, CustomTemplateType

# model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
# # model_type = ModelType.llama3_70b_instruct_int4

# llm_engine = get_vllm_engine(
#     model_type, 
#     torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
#     tensor_parallel_size=2,
#     # max_model_len=4096,
#     # gpu_memory_utilization=0.95,
#     engine_kwargs = {
#         # "enforce_eager": True,
#         "max_num_seqs": 128,
#         "seed": 42,
#     }
# )

# template_type = get_default_template_type(model_type)
# template = get_template(template_type, llm_engine.hf_tokenizer)



In [1]:
prompt = f"Please answer the following question in a short sentence! Here are 3 sentences that are incomplete due to truncation: \n\
incomplete sentences 1: la does not cause a positive COVID-19 test\n\
incomplete sentences 2: Dilma Rousseff stated\n\
incomplete sentences 3: The text claims that mass media owners, called Satanists in the text, are forcing the public to voluntarily look for a coronavirus vaccine. It also describes the vaccine (which still does not exist) as\n\
Please judge whether the following sentence is truncated like any of the above sentences. If it is truncated like the above sentences, then just answer \"Yes\", otherwise just answer \"No\":\n{1}"
print(prompt)

Please answer the following question in a short sentence! Here are 3 sentences that are incomplete due to truncation: 
incomplete sentences 1: la does not cause a positive COVID-19 test
incomplete sentences 2: Dilma Rousseff stated
incomplete sentences 3: The text claims that mass media owners, called Satanists in the text, are forcing the public to voluntarily look for a coronavirus vaccine. It also describes the vaccine (which still does not exist) as
Please judge whether the following sentence is truncated like any of the above sentences. If it is truncated like the above sentences, then just answer "Yes", otherwise just answer "No":
1


In [4]:
generation_config = VllmGenerationConfig(
    max_new_tokens=512,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True,
)
data_train = load_train()
prompt_list = [
    # "which is thelongest river"
]
for item in data_train:
    claim = item["claim"]
    prompt = f"Please answer the following question in a short sentence! Here are 3 sentences that are incomplete due to truncation: \n\
incomplete sentences 1: la does not cause a positive COVID-19 test\n\
incomplete sentences 2: Dilma Rousseff stated\n\
incomplete sentences 3: The text claims that mass media owners, called Satanists in the text, are forcing the public to voluntarily look for a coronavirus vaccine. It also describes the vaccine (which still does not exist) as\n\
Please judge whether the following sentence is truncated like any of the above sentences. If it is truncated like the above sentences, then just answer \"Yes\", otherwise just answer \"No\":\n{claim}"
    prompt_list.append(prompt)


request_list = [{'query': prompt} for prompt in prompt_list]
resp_list = get_resp_list(request_list)
resp_list = [i["response"] for i in resp_list]

for i, resp in enumerate(resp_list):
    # print(prompt_list[i])
    if resp.lower().startswith("yes"):
        print(f'{data_train[i]["id"]}: {data_train[i]["claim"]}')
    # print(f'{data_train[i]["id"]}: {resp.strip()}')


100%|██████████| 14384/14384 [18:27<00:00, 12.99it/s]

1: No. This sentence is not truncated and provides a complete thought.
2: No, this sentence is not truncated. It contains a complete thought and does not require additional information to be understood.
5: Yes, the sentence is also truncated like the incomplete sentences provided. The claim about the increase in COVID-19 mortality by 36% due to influenza vaccines is not supported by scientific evidence.
9: No, the sentence is not truncated.
11: No, this sentence is not truncated like the given examples. It is a title with two separate topics.
12: No, this sentence is not truncated like the previous examples. It contains a complete thought and does not require additional information to be understood.
13: No.
14: No, this sentence is not truncated like the given examples. It is a complete sentence describing video footage.
15: No, the sentence is complete. It conveys a clear and coherent meaning.
16: No, this sentence is not truncated like the previous examples. It is a complete sentence

查看Claim改变的数量

In [2]:
data_train = load_train()
data_train_bak = load_train_bak()
data_search = load_search()

cnt = 0
for i, item in enumerate(data_train):

    data_search[i]["label"] = item["label"]
    data_search[i]["claim"] = item["claim"]
    # if item["claim"] != data_train_bak[i]["claim"]:
    #    cnt += 1

cnt


0

In [1]:
data_llm = covmis.load_train_llm()

def get_lst_sent(s):

    for x in ["References:", "Sources:"]:
        src_pos = s.find(x)
        if src_pos != -1:
            s = s[:src_pos].strip()

    pos = s.rfind("\n")
    return s[pos + 1:]

for item in data_llm[:10]:
    print(get_lst_sent(item["prior_knowledge_llama3_v1_K=5"]))


NameError: name 'covmis' is not defined

In [2]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

DEFAULT_SEED = 42
import numpy as np
import random

# def set_seed(seed=DEFAULT_SEED):
#     np.random.seed(seed)
#     random.seed(seed)
# set_seed()

data_train = load_train()
data_train_bak = load_train_bak()

label_old = {}
for item in data_train_bak:
    label_old[item["id"]] = item["label"]


data_train_new = []
for item in data_train:
    if item["label"] != 1:
        data_train_new.append(item)

train_list, test_list = train_test_split(data_train_new, test_size=0.2, random_state=DEFAULT_SEED, shuffle=True)
valid_list, test_list = train_test_split(test_list, test_size=0.5, random_state=DEFAULT_SEED, shuffle=True)

cnt = 0
labels, preds = [], [] # preds为covmis原始标签
for item in test_list:
    if int(label_old[item["id"]]) == 2:
        preds.append(1)
    elif int(label_old[item["id"]]) == 0:
        preds.append(0)
    else:
        raise Exception()

    if item["label"] == 2:
        labels.append(1)
    elif item["label"] == 0:
        labels.append(0)
    else:
        
        raise Exception()
    
    if int(label_old[item["id"]]) != item["label"]:
        cnt += 1

acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='macro')
precision = precision_score(labels, preds, average='macro')
recall = recall_score(labels, preds, average='macro')

acc, f1, precision, recall


(0.9254098360655738, 0.8873715531505428, 0.953031719265943, 0.8494565217391303)

In [3]:
# save_search(data_search)
